![title](DEP_LOGO_2.jpg)

# Trend Station Water Quality Data 
## Web Application Demo

In [1]:
#HIDDEN

import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import csv
from geopy.geocoders import Nominatim
from time import sleep
import folium
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import seaborn as sns
from scipy import stats

In [2]:
#HIDDEN

dfhead = pd.read_csv('gm_export_TS_Location.csv')
dfhead = dfhead.fillna(0)
dfheadloc = dfhead[dfhead.latitude != 0]

geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Flatwoods, WV")

longitude = location.longitude
latitude = location.latitude

# create map of West Virginia using latitude and longitude values
ts_locations = folium.Map(location=[latitude, longitude], tiles="OpenStreetMap", zoom_start=7.4)

# add markers to map
for lat, lng, label in zip(dfheadloc['latitude'], dfheadloc['longitude'], dfheadloc['station_id']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=True).add_to(ts_locations)  

ts_locations

## Trend Station Location

In [3]:
#HIDDEN

dfhead = pd.read_csv('gm_export_TS_Location.csv')
dfhead = dfhead.fillna(0)
dfheadloc = dfhead[dfhead.latitude != 0]


def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

dropdown_stations_id = widgets.Dropdown(options = unique_sorted_values(dfheadloc.station_id), description='Trend Station')

output_two = widgets.Output()



def common_filtering(station):
    
    output_two.clear_output()
    
    
    global common_filter
    common_filter = dfheadloc[(dfheadloc.station_id == station)]
    
    with output_two:
        display(common_filter)
        
    
        
def dropdown_stations_id_eventhandler(change):
    common_filtering(change.new)

    
dropdown_stations_id.observe(dropdown_stations_id_eventhandler, names='value')
    
display(dropdown_stations_id)


Dropdown(description='Trend Station', options=('TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006', 'TS007', …

In [4]:
#HIDDEN
display(output_two)

Output()

## Trend Station Data

In [75]:
#HIDDEN

df_comb = pd.read_csv('TS_New_Combined_Final.csv')
df_comb = df_comb.fillna(0)
#df_comb['test_batch_id'] = pd.to_datetime(df_comb['test_batch_id'], format="%m/%d/%Y")
df_comb['result_value'] = pd.to_numeric(df_comb['result_value'])
df_comb = df_comb.sort_values(by=['station_id', 'test_batch_id'])
#df_comb = df_comb.set_index('test_batch_id')

ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

dropdown_stations = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_comb.station_id), description='Trend Station')
dropdown_param = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_comb.param_name), description='Parameter')

output_three = widgets.Output()
plot_output = widgets.Output()


def common_filtering1(station, param):
    
    output_three.clear_output()
    plot_output.clear_output()
    
    if (station == ALL) & (param == ALL):
        global common_filter1
        common_filter1 = df_comb
    elif (station == ALL):
        common_filter1 = df_comb[df_comb.param_name == param]
    elif (param == ALL):
        common_filter1 = df_comb[df_comb.station_id == station]
    else:
        common_filter1 = df_comb[(df_comb.station_id == station) & (df_comb.param_name == param)]
    
    with output_three:
        display(common_filter1)
        
    with plot_output:
        if param == ALL or station == ALL:
            print("No Available Plots for the Selected Station/Parameter Combination!")
        else:
            plt.figure(figsize=(20,8));
            plt.plot(common_filter1['result_value'], 'go--', linewidth=1, markersize=6);
            plt.grid();
            plt.ylabel(param, fontsize=15);
            plt.xlabel('Date', fontsize=15);
            plt.title(station, fontsize=25);
            plt.show();


def dropdown_stations_eventhandler(change):
    common_filtering1(change.new, dropdown_param.value)
    
def dropdown_param_eventhandler(change):
    common_filtering1(dropdown_stations.value, change.new)
    
dropdown_stations.observe(dropdown_stations_eventhandler, names='value')
dropdown_param.observe(dropdown_param_eventhandler, names='value')
    
display(dropdown_stations)
display(dropdown_param)

Dropdown(description='Trend Station', options=('ALL', 'TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006', 'T…

Dropdown(description='Parameter', options=('ALL', 'ALKALINITY', 'ALUMINUM', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM'…

In [76]:
#HIDDEN
display(output_three)

Output()

In [26]:
#HIDDEN
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
display(plot_output)

Output(outputs=({'output_type': 'stream', 'text': 'No Available Plots for the Selected Station/Parameter Combi…

## Trend Station Statistics

Need to convert DataFrame structure prior to this step!

In [86]:
group = common_filter1.groupby('param_name')
df2 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

df2.reset_index(inplace=True)
df2 = df2.rename({0: 'DATES'}, axis='columns')
df2

,param_name,DATES
0,ALKALINITY,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
1,ALUMINUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
2,ANTIMONY,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
3,ARSENIC,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
4,BERYLLIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
5,BI-CARBONATE,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
6,CADMIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
7,CALCIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
8,CARBONATE,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."
9,CHROMIUM,"[1/1/2016, 1/1/2017, 1/1/2018, 10/1/2016, 10/1..."


In [36]:
param_dates = df2[df2.param_name == 'BERYLLIUM'].DATES.item()
param_dates[5]

'4/1/2017'

In [37]:
for i in param_dates:
    print(i)

1/1/2016
4/1/2016
8/1/2016
10/1/2016
1/1/2017
4/1/2017
10/1/2017
1/1/2018
4/1/2018


In [128]:
ts = 'TS185'

df_comb = pd.read_csv('TS_New_Combined_Final.csv')
df_comb = df_comb.fillna(0)


group = df_comb.groupby('param_name')
df2 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

df2.reset_index(inplace=True)
df2 = df2.rename({0: 'DATES'}, axis='columns')

dates = df2[df2.param_name == 'BERYLLIUM'].DATES.item()

initial_date = dates[0]

df_comb_date = common_filter1[common_filter1['station_id'].str.contains(ts, na=False)]
df_comb_date = df_comb_date[df_comb_date['test_batch_id'] == initial_date]
df_comb_date = df_comb_date.drop(['station_id', 'sample_id', 'fraction', 'result_unit', 'test_batch_id'], axis=1)
df_comb_date = df_comb_date.set_index('param_name').transpose()

df_comb_date = df_comb_date.astype('float')
df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB ph', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']

df_comb_concat = []
df_comb_concat = pd.concat([df_comb_date])

as_list = df_comb_concat.index.tolist()
idx = as_list.index('result_value')
as_list[idx] = initial_date
df_comb_concat.index = as_list

print("Total Number of Records = ")

for i, date in enumerate(dates):
    
    df_comb_date = common_filter1[common_filter1['station_id'].str.contains(ts, na=False)]
    df_comb_date = df_comb_date[df_comb_date['test_batch_id'] == initial_date]
    df_comb_date = df_comb_date.drop(['station_id', 'sample_id', 'fraction', 'result_unit', 'test_batch_id'], axis=1)
    df_comb_date = df_comb_date.set_index('param_name').transpose()
    
    #df_comb_date = df_comb_date.astype('float')

    df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB ph', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']
    
    df_comb_concat = pd.concat([df_comb_concat, df_comb_date])
    
    as_list = df_comb_concat.index.tolist()
    idx = as_list.index('result_value')
    as_list[idx] = dates[i]
    df_comb_concat.index = as_list

    df_comb_concat.index.name = 'TS185 - Horseshoe Run'
    
    print(1+(i+1))
    
df_comb_concat = df_comb_concat.fillna(0)
    
df_comb_concat = df_comb_concat.loc[:, (df_comb_date != 0).any(axis=0)]

df_comb_concat.index = pd.to_datetime(df_comb_concat.index)
  
df_comb_concat

Total Number of Records = 
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


,FIELD ph,FIELD CONDUCTIVITY,TEMPERATURE,FLOW,LAB CONDUCTIVITY,DISSOLVED SOLIDS,LAB ph,ALKALINITY,IRON_T,ALUMINUM_T,MERCURY,SELENIUM,ZINC,NITRATE,SODIUM,CALCIUM,MAGNESIUM,POTASSIUM,BI-CARBONATE,PHOSPHATE
TS185 - Horseshoe Run,,,,,,,,,,,,,,,,,,,,
2016-01-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-01-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-02-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-03-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-04-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-05-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-07-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-08-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
2016-10-01,8.1,78.0,0.6,8.86,68.0,34.0,6.43,26.0,0.06,0.05,1.0,0.78,9.0,5.12,5.46,4.0,1.46,1.07,6.0,0.25
